#KNN Method

In [ ]:
import numpy as np

X_train = np.load('kmnist-train-imgs.npz')['arr_0']
Ytrain = np.load('kmnist-train-labels.npz')['arr_0']
X_test = np.load('kmnist-test-imgs.npz')['arr_0']
Ytest = np.load('kmnist-test-labels.npz')['arr_0']

#Now the arrays are 60.000 images 28x28. We want it to be 60.000 x 784.

Xtrain = X_train.reshape(X_train.shape[0], -1)
Xtest = X_test.reshape(X_test.shape[0], -1)

#Xtrain = np.file("kminst-train-imgs.npz")
#Ytrain = np.file("kmnist-train-labels.npz")
#Xtest = "kmnist-test-imgs.npz"
#Ytest = "kmnist-test-labels.npz"

print(X_train.shape, X_test.shape)
print(Xtrain.shape,Ytrain.shape)
print(Xtest.shape, Ytest.shape)

(60000, 28, 28) (10000, 28, 28)
(60000, 784) (60000,)
(10000, 784) (10000,)


In [ ]:
""" Divide Training set """
from sklearn.model_selection import train_test_split

testsize = 0.1 #percentuale messa in validation
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain, Ytrain, test_size=testsize, random_state=42)

print("Training set shapes" , Xtrain.shape, Ytrain.shape)
print("Validation set shapes" ,Xval.shape, Yval.shape)

Training set shapes (54000, 784) (54000,)
Validation set shapes (6000, 784) (6000,)


In [ ]:
def knn_inference(X, Xtrain, Ytrain, k):
    classes = Ytrain.max() + 1
    D = _dist_matrix(X, Xtrain)
    neighs = np.argpartition(D, k, 1)[:, :k]
    counts = _bincount_rows(Ytrain[neighs], classes)
    labels = np.argmax(counts, 1)
    return labels

def _bincount_rows(X, values):
    """Compute one histogram for each row."""
    # np.bincount works only on 1D arrays.
    # This extends it to 2D arrays.
    m = X.shape[0]
    idx = X.astype(int) + values * np.arange(m)[:, np.newaxis]
    c = np.bincount(idx.ravel(), minlength=values * m)
    return c.reshape(-1, values)

def _dist_matrix(X1, X2):
    """Compute the matrix of all squared distances."""
    Q1 = (X1 ** 2).sum(1, keepdims=True)
    Q2 = (X2 ** 2).sum(1, keepdims=True)
    return Q1- 2 * X1 @ X2.T + Q2.T


def knn_select_k(X, Y, maxk=101):
    """Leave-one-out selection of the number of neighbors."""
    D = _dist_matrix(X, X)
    classes = Y.max() + 1
    np.fill_diagonal(D, np.inf)
    neighs = np.argsort(D, 1)
    best_k = 1
    best_acc =-1
    for k in range(1, maxk + 1):
        counts = _bincount_rows(Y[neighs[:, :k]], classes)
        labels = np.argmax(counts, 1)
        accuracy = (labels == Y).mean()
        if accuracy > best_acc:
            best_acc = accuracy
            best_k = k
    return best_k

In [ ]:
#bestk = 4
bestk = knn_select_k(Xtrain, Ytrain)
print("Best k value: ", bestk)

predictions = knn_inference(Xtrain, Xtrain, Ytrain, bestk)
Tpredictions = knn_inference(Xtest, Xtrain, Ytrain, bestk)

accuracy = (predictions == Ytrain).mean()
Taccuracy = (Tpredictions == Ytest).mean()

print("Training accuracy:", accuracy * 100)
print("Test accuracy:", Taccuracy * 100)